In [2]:
import requests
API_KEY = "f8363732-1217-4405-85bb-e8d23da98f1d"

In [3]:
def get_tgt(api_key):
    url = "https://utslogin.nlm.nih.gov/cas/v1/api-key"
    params = {'apikey': api_key}
    response = requests.post(url, data=params)
    tgt_url = response.headers['location']
    return tgt_url

In [4]:
def get_st(tgt):
    response = requests.post(tgt, data={'service': 'http://umlsks.nlm.nih.gov'})
    return response.text


In [5]:
def search_umls(symptom, st):
    base_url = "https://uts-ws.nlm.nih.gov/rest/search/current"
    params = {
        "string": symptom,
        "ticket": st
        }
    response = requests.get(base_url, params=params)
    data = response.json()
    results = data['result']['results']
    if results:
        return results[0]['ui']  # The CUI
    return None

In [6]:
def get_related_diseases(cui, st):
    url = f"https://uts-ws.nlm.nih.gov/rest/content/current/CUI/{cui}/relations"
    params = {"ticket": st}
    response = requests.get(url, params=params)
    data = response.json()
    diseases = []

    for rel in data.get('result', []):
        if "may_be_diagnosed_by" in rel.get("relationLabel", "").lower() or \
           "associated_with" in rel.get("relationLabel", "").lower():
            diseases.append(rel['relatedIdName'])
    return diseases

In [10]:
%pip install numpy==1.26.4 --force-reinstall --no-cache-dir

   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
    --------------------------------------- 0.3/15.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/15.5 MB 1.7 MB/s eta 0:00:09
   -- ------------------------------------- 1.0/15.5 MB 1.9 MB/s eta 0:00:08
   ---- ----------------------------------- 1.6/15.5 MB 2.2 MB/s eta 0:00:07
   ------ --------------------------------- 2.4/15.5 MB 2.5 MB/s eta 0:00:06
   -------- ------------------------------- 3.1/15.5 MB 2.7 MB/s eta 0:00:05
   ---------- ----------------------------- 3.9/15.5 MB 2.9 MB/s eta 0:00:04
   ------------ --------------------------- 5.0/15.5 MB 3.1 MB/s eta 0:00:04
   -------------- ------------------------- 5.8/15.5 MB 3.2 MB/s eta 0:00:04
   ----------------- ---------------------- 6.8/15.5 MB 3.4 MB/s eta 0:00:03
   -------------------- ------------------- 7.9/15.5 MB 3.6 MB/s eta 0:00:03
   ------------------------ --------------- 9.4/15.5 MB 3.9 MB/s eta 0:00:02
   ----------

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.3 which is incompatible.
streamlit 1.37.1 requires pillow<11,>=7.1.0, but you have pillow 11.2.1 which is incompatible.


In [1]:
import numpy
print(numpy.__version__)  

1.26.4


In [ ]:
import pandas as pd

df = pd.read_csv(r"C:\Users\Saswat Subhadarsi\Desktop\Project\Asclepius\Data\Medical-data.csv")

tgt = get_tgt(API_KEY)  
results = []

for symptom_str in df["symptoms"]:
    st = get_st(tgt)  
    cui = search_umls(symptom_str, st)  
    if cui and cui != "NONE":
        diseases = get_related_diseases(cui, st) #CUI
        results.append({
            "symptoms": symptom_str,
            "diseases": "; ".join(diseases) if diseases else "Not Found"
        })
    else:
        results.append({
            "symptoms": symptom_str,
            "diseases": "Not Found"
        })

# Save all results to CSV
pd.DataFrame(results).to_csv("symptom_disease.csv", index=False)


KeyboardInterrupt: 